In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv('data/letter-recognition.csv')

In [11]:
df.head()

,lettr,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [14]:
df.describe()

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [18]:
# Förbered data
# Vi väljer relevanta kolumner. 'car_or_bus' är vår target (det vi vill klassificera).
# Vi tar bort 'id', 'id_android' och 'linha' (för många saknade värden).
chosen_letters = ['A','B','C','Z']
df_filtred = df[df['lettr'].isin(chosen_letters)]

X = df_filtred.drop(columns=['lettr'])
y = df_filtred['lettr']

In [19]:
def run_svm_experiment(experiment_name, X_data, y_data, test_size, kernel, scale_data=False):
    print(f"\n{'='*10} {experiment_name} {'='*10}")
    
    # Split data random_state=42
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=test_size,)
    
    # Normalisering (skalning)
    if scale_data:
        scaler = MinMaxScaler() # Skalar till intervallet [0, 1]
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        print("Data: Normaliserat (MinMax)")
    else:
        print("Data: Icke-normaliserat (Original)")
        
    print(f"Split: Train={(1-test_size)*100:.0f}%, Test={test_size*100:.0f}%")
    print(f"Kernel: {kernel}")

    # Träna modell
    svc = SVC(kernel=kernel)
    svc.fit(X_train, y_train)
    
    # Prediktion och utvärdering
    y_pred = svc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    print(f"Accuracy: {acc:.4f}")
    print("Confusion Matrix:")
    print(cm)
    return acc, cm

# --- EXPERIMENT ---
# a.i: Icke-normaliserat, 75% train / 25% test, RBF (default)
run_svm_experiment("Exp 1: Oskalat Data", X, y, 0.25, 'rbf', scale_data=False)

# a.ii: Normaliserat, 75% train / 25% test, RBF
run_svm_experiment("Exp 2: Normaliserat Data", X, y, 0.25, 'rbf', scale_data=True)

# b.ii: Normaliserat, 50% train / 50% test, RBF (Annan split)
run_svm_experiment("Exp 3: Annan Split (50/50)", X, y, 0.75, 'rbf', scale_data=True)

# c: Normaliserat, 75% train / 25% test, Linear (Annan kernel)
run_svm_experiment("Exp 4: Linjär Kernel", X, y, 0.25, 'linear', scale_data=True)


========== Exp 1: Oskalat Data ==========
Data: Icke-normaliserat (Original)
Split: Train=75%, Test=25%
Kernel: rbf
Accuracy: 0.9960
Confusion Matrix:
[[200   0   1   0]
 [  1 182   0   0]
 [  0   1 200   0]
 [  0   0   0 172]]

========== Exp 2: Normaliserat Data ==========
Data: Normaliserat (MinMax)
Split: Train=75%, Test=25%
Kernel: rbf
Accuracy: 0.9987
Confusion Matrix:
[[191   0   1   0]
 [  0 223   0   0]
 [  0   0 187   0]
 [  0   0   0 155]]

========== Exp 3: Annan Split (50/50) ==========
Data: Normaliserat (MinMax)
Split: Train=25%, Test=75%
Kernel: rbf
Accuracy: 0.9912
Confusion Matrix:
[[596   0   6   0]
 [  5 569   0   0]
 [  2   1 536   0]
 [  1   5   0 548]]

========== Exp 4: Linjär Kernel ==========
Data: Normaliserat (MinMax)
Split: Train=75%, Test=25%
Kernel: linear
Accuracy: 0.9881
Confusion Matrix:
[[205   2   3   1]
 [  1 197   0   0]
 [  1   0 192   0]
 [  0   1   0 154]]


(0.988110964332893,
 array([[205,   2,   3,   1],
        [  1, 197,   0,   0],
        [  1,   0, 192,   0],
        [  0,   1,   0, 154]]))